In [42]:
from PIL import Image
import torch
import utils
from torchvision import models
import numpy as np
from PIL import Image

model_path = r"C:\rkka_Projects\cell_death_v2\trained_models\test_5_classes_22.032991_0.9728_sota.pth"
path = r"C:\rkka_Projects\cell_death_v2\Data\model_training\mip\test\0_TreatedAlive\230510.HeLa_TNF (12)_patch_6_t_0.png"

# Load Model
model = models.resnet101(pretrained=True)
num_features = model.fc.in_features
model.fc = torch.nn.Sequential(
    torch.nn.Dropout(0.2),
    torch.nn.Linear(num_features, 5)
)
model.load_state_dict(torch.load(model_path))
model = model.cuda()

transform = models.ResNet101_Weights.IMAGENET1K_V2.transforms()
image = Image.open(path).convert('L')
image = np.array(image) 

image = torch.from_numpy(image)

outputs = []
labels = []
with torch.no_grad():
    model = model.eval()
    image = image.unsqueeze(0).repeat(3,1,1).float()
    image = transform(image)
    output = model(image.unsqueeze(0).cuda())
    outputs.append(output)
    labels.append(int(torch.max(output, 1)[1].cpu().numpy()[0]))

In [43]:
import torch.nn.functional as F
with torch.no_grad():
    # output = torch.from_numpy(wanted_probabilities[0])
    # Apply softmax to get probabilities
    probabilities = []
    for output in outputs:
        probs = F.softmax(output.cpu(), dim=1)  # shape: [1, 5]
        # If you want it as a simple list or tensor
        probs = probs.squeeze()  # shape: [5]
        probs = probs.numpy()
        probabilities.append(probs)
        
probs

array([0.0048331 , 0.011102  , 0.00635452, 0.00380736, 0.97390294],
      dtype=float32)